***Google Service Account| Authentication***

In [ ]:
#!pip install gspread oauth2client

In [ ]:
!gdown '1Ii5JdOx_Z0BlIgqJw9AAYON09wMWcnTq'

Downloading...
From: https://drive.google.com/uc?id=1Ii5JdOx_Z0BlIgqJw9AAYON09wMWcnTq
To: /content/gsheet.json
100% 2.39k/2.39k [00:00<00:00, 10.4MB/s]


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Load credentials from the JSON key file (replace 'path/to/credentials.json' with your actual path)
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/gsheet.json', scope)
client = gspread.authorize(creds)

**Read Google Sheet**

In [ ]:
from gspread.models import Worksheet
# Open the Google Sheet using its sheet_id
spreadsheet = '1geJdStSt1dl9GFdod27q3qi30VNDlNa-WefCpVxmTcY'
sheet = client.open_by_key(spreadsheet)
worksheet = sheet.get_worksheet(0)

***Convert Google Sheets Data to: Pandas Dataframe***

In [ ]:
import pandas as pd

In [ ]:
# Get all values from the worksheet
values = worksheet.get_all_values()

# Convert the values to a Pandas DataFrame
bd_mobile_price = pd.DataFrame(values[1:], columns=values[0])  # Assuming the first row contains column headers

In [ ]:
bd_mobile_price.tail()

,mobile_marketing_name,device,price_bdt
1517,A3,A3,14990
1518,A31,A31,22990
1519,A33,A33,24990
1520,A36,A36,26990
1521,A3s,A3s,16990


In [ ]:
# Define the price range categories
price_ranges = [
    ("basic", 0, 10000),
    ("lower mid", 10000, 15000),
    ("mid", 15000, 25000),
    ("upper mid", 25000, 40000),
    ("premium", 40000, 60000),
    ("flagship", 60000, float('inf'))
]

In [ ]:
# Function to categorize prices
def categorize_price(price):
    for category, lower, upper in price_ranges:
        if lower <= price < upper:
            return category
    return "Unknown"

# Clean the 'price_bdt' column by converting to numeric and handling non-convertible values
bd_mobile_price['price_bdt'] = pd.to_numeric(bd_mobile_price['price_bdt'], errors='coerce')
# Apply the categorize_price function to create a new column
bd_mobile_price['price_category'] = bd_mobile_price['price_bdt'].apply(categorize_price)

In [ ]:
# Print the updated DataFrame
bd_mobile_price

,mobile_marketing_name,device,price_bdt,price_category
0,A46,A46,18990,mid
1,A47,A47,21990,mid
2,A48,A48,24990,mid
3,A49,A49,27990,upper mid
4,A49 Play,A49 Play,31990,upper mid
...,...,...,...,...
1517,A3,A3,14990,lower mid
1518,A31,A31,22990,mid
1519,A33,A33,24990,mid
1520,A36,A36,26990,upper mid


***Write to the same worksheet***

In [ ]:
# Update the Google Sheet with the new 'price_category' column
price_categories = bd_mobile_price['price_category'].tolist()
worksheet.update('D1', 'price_category')  # Assuming the 'price_category' column will be in column D
worksheet.update('D2:D' + str(len(price_categories) + 1), [[category] for category in price_categories])

print("Price categories added as a new column in the Google Sheet.")

Price categories added as a new column in the Google Sheet.


***Doing Science.....***